In [171]:
# coding: utf-8
import pandas as pd
from pathlib import Path
import json
root = Path('.')
root = root.absolute()
test_cases_root = root / "testcases"
results_root = root / "fmcad_results_2023-05-16_23-03-02/"

In [172]:
print("root: ", root)
print("test_cases_root: ", test_cases_root)
print("results_root: ", results_root)
# cwd
print("cwd: ", Path.cwd())

root:  /home/eytan/Work/TheSy/experiments/cvc4_newth_alltogether
test_cases_root:  /home/eytan/Work/TheSy/experiments/cvc4_newth_alltogether/testcases
results_root:  /home/eytan/Work/TheSy/experiments/cvc4_newth_alltogether/fmcad_results_2023-05-16_23-03-02
cwd:  /home/eytan/Work/TheSy/experiments/cvc4_newth_alltogether


In [173]:
test_cases = [y for x in test_cases_root.iterdir() for y in x.iterdir() if y.suffix == ".th"]
rel_cases = [x.relative_to(test_cases_root) for x in test_cases]
configs = [x.name for x in results_root.iterdir() if x.is_dir()]
jsons = {n: pd.read_csv(results_root /  n / "stats.csv") for n in configs}
# Rename columns such that Unnamed: 0 is the test suite nameL
jsons = {n: j.rename(columns={"Unnamed: 0": "test_suite"}) for n, j in jsons.items()}
# Assert test suite column of first configurations contains at least one case of clam
assert any(jsons[configs[0]]["test_suite"].str.contains("clam"))
for n, j in jsons.items():
    j["relative_path"] = j.apply(lambda x: (Path(x["test_suite"]) / x['file_name'].replace('stats.json', 'th')), axis=1)

In [174]:
for n, j in jsons.items():
    fixed_config = n.replace('case_split_proof_split_', '')
    split_depth = int(fixed_config[0])
    features = fixed_config[2:]
    j['split_depth'] = split_depth
    j['features'] = features

In [175]:
from numpy import NaN


colored_stats = {}
# Calculate relative_path for each test case
for n, j in jsons.items():
    # From .at
    j["num_terms_added"] = 0
    def get_at(row):
        at_path = results_root / n / row['relative_path'].with_suffix(".at")
        if at_path.exists():
            with at_path.open() as f:
                return len(f.readlines())
        else:
            return None
    j["num_terms_added"] = j.apply(get_at, axis=1)
    # From stats.json
    j["total_search_time"] = NaN
    j['total_apply_time'] = NaN
    j['total_rebuild_time'] = NaN
    def get_stats(row):
        stats_path = results_root / n / row['relative_path'].with_suffix(".stats.json")
        if stats_path.exists():
            with stats_path.open() as f:
                return json.load(f)
        else:
            return None
    stats = j.apply(get_stats, axis=1)
    j['total_search_time'] = stats.apply(lambda s: s['total_search_time'])
    j['total_apply_time'] = stats.apply(lambda s: s['total_apply_time'])
    j['total_rebuild_time'] = stats.apply(lambda s: s['total_rebuild_time'])

    # From colored_stats.json
    j["total_enodes"] = NaN
    j["num_splits"] = 0
    j["should_delete"] = NaN
    j["deleted_colored_enodes"] = NaN
    j["colored_unions_count"] = NaN
    j['total_colors_size'] = NaN
    j['black_size'] = NaN
    j['vacuos_colors'] = NaN
    j['split_sizes'] = NaN
    j['graph_memory'] = NaN
    j['eclasses_count'] = NaN
    j['id_count'] = NaN
    j['colored_unions_count'] = NaN
    j['deleted_colored_enodes'] = NaN
    j['colored_equivalences_size'] = NaN
    j['black_colored_classes_size'] = NaN
    j['parent_color_tracking_size'] = NaN
    

    # Find all existing colored_stats.json files by iterating over j['relative_path']
    def get_max(row):
        x = row['relative_path']
        # Find all colored_stats.json files
        cs_path = results_root / n / x.with_suffix(".colored_stats.json")
        if cs_path.exists():
            with cs_path.open() as f:
                cs = json.load(f)
                colored_stats[(n, x)] = cs
            filtered = [x[1] for x in cs]
            for stats in filtered:
                stats['total_enodes'] = stats['black_size']
                if 'colors_sizes' in stats:
                    stats['total_enodes'] += sum(stats['colors_sizes'].values())
                if 'split_sizes' in stats:
                    stats['total_enodes'] += sum(stats['split_sizes'])
            # take max of total_enodes from filtered and add it to j
            return max(filtered, key=lambda x: x['total_enodes'])
        else:
            return None
        
    def collect_num_splits(stats):
        if 'colors_sizes' in stats:
            return len(stats['colors_sizes'])
        elif 'split_sizes' in stats:
            return len(stats['split_sizes'])
        else:
            return 0
        
    max_stats = j.apply(get_max, axis=1)
    
    # do by arity 
        # by_arity: IndexMap<usize, usize>,
    j['total_enodes'] = max_stats.apply(lambda s: s['total_enodes'])
    j['num_splits'] = max_stats.apply(collect_num_splits)
    j['should_delete'] = max_stats.apply(lambda s: sum(s['should_delete'].values()) if 'should_delete' in s else 0)
    j['deleted_colored_enodes'] = max_stats.apply(lambda s: s['deleted_colored_enodes'])
    j['colored_unions_count'] = max_stats.apply(lambda s: s['colored_unions_count'])
    j['total_colors_size'] = max_stats.apply(lambda s: sum([v for k, v in s['colors_sizes'].items()]) if 'colors_sizes'in s else 0)
    j['black_size'] = max_stats.apply(lambda s: s['black_size'])
    j['vacuos_colors'] = max_stats.apply(lambda s: len(s['vacuos_colors']) if 'vacuos_colors' in s else 0)
    j['split_sizes'] = max_stats.apply(lambda s: sum(s['split_sizes']) if 'split_sizes' in s else 0)
    j['graph_memory'] = max_stats.apply(lambda s: s['graph_memory'])
    j['eclasses_count'] = max_stats.apply(lambda s: s['eclasses_count'])
    j['id_count'] = max_stats.apply(lambda s: s['id_count'])
    j['colored_unions_count'] = max_stats.apply(lambda s: s['colored_unions_count'])
    j['deleted_colored_enodes'] = max_stats.apply(lambda s: s['deleted_colored_enodes'])
    j['colored_equivalences_size'] = max_stats.apply(lambda s: s['colored_equivalences_size'])
    j['black_colored_classes_size'] = max_stats.apply(lambda s: s['black_colored_classes_size'])
    j['parent_color_tracking_size'] = max_stats.apply(lambda s: s['parent_color_tracking_size'])

In [176]:
# Start adding missing files by first adding an error column
for j in jsons.values():
    j["error"] = None

# Now add the missing files. For each missing case find the corresponding error file
for n, j in jsons.items():
    # Find missing cases by relative path to rel_cases
    in_df = j['relative_path'].unique()
    missing_cases = [r for r in rel_cases if r not in in_df]
    for fn in missing_cases:
        errors = (results_root / n / fn.with_suffix('.err')).read_text()
        j.loc[len(j)] = {"relative_path": fn, "error": errors}
        

In [177]:
for c in configs:
    if not ('keep' in c or 'colored' in c):
        continue
    # Filter errors by x is not None and ('Timeout' in x or 'memory' in x)
    temp = jsons[c]['error'].apply(lambda x: x is not None and not ('Timeout' in x or 'memory' in x))
    print(c)
    break
jsons[c][temp]['error'].value_counts()


case_split_proof_split_3_split_clone_keep_splits_stats


error
07:39:23 [ERROR] Unrecognized token `=>` found at 3771:3773\nExpected one of "(", "=", "?", r#"#[^#]+#"# or r#"[a-zA-Z+\\-/_*&$%^@0-9]+"#\nthread 'main' panicked at 'Please implement error handling', src/thesy/semantics.rs:123:17\nnote: run with `RUST_BACKTRACE=1` environment variable to display a backtrace\n                18
thread 'main' panicked at 'called `Result::unwrap()` on an `Err` value: UnrecognizedToken { token: (1, Token(13, "=>"), 3), expected: ["\"(\"", "\"=\"", "\"?\"", "r#\"#[^#]+#\"#", "r#\"[a-zA-Z+\\\\-/_*&$%^@0-9]+\"#"] }', src/bin/expl_experiment.rs:127:34\nnote: run with `RUST_BACKTRACE=1` environment variable to display a backtrace\n    15
07:39:19 [ERROR] Unrecognized token `=>` found at 1247:1249\nExpected one of "(", "=", "?", r#"#[^#]+#"# or r#"[a-zA-Z+\\-/_*&$%^@0-9]+"#\nthread 'main' panicked at 'Please implement error handling', src/thesy/semantics.rs:123:17\nnote: run with `RUST_BACKTRACE=1` environment variable to display a backtrace\n             

In [178]:
j

,test_suite,Unnamed: 1,time,stop_reasons,success,lemma_count,proofs_later_filtered,case_split_root_count,case_split_had_vacuity,total_allocated,...,black_size,vacuos_colors,split_sizes,graph_memory,eclasses_count,id_count,colored_equivalences_size,black_colored_classes_size,parent_color_tracking_size,error
0,isaplanner,0.0,7.052348,[],0.0,0.0,0.0,0.0,0.0,8.306160e+09,...,903.0,0.0,0.0,3979244.0,842.0,3919.0,16882.0,385.0,0.0,None
1,isaplanner,1.0,4.175650,[],0.0,0.0,0.0,0.0,0.0,7.078351e+09,...,753.0,0.0,0.0,2853108.0,673.0,2064.0,19462.0,215.0,76.0,None
2,isaplanner,2.0,0.002470,[],1.0,0.0,0.0,0.0,0.0,9.964813e+06,...,55.0,0.0,0.0,36868.0,52.0,56.0,10.0,1.0,0.0,None
3,isaplanner,3.0,0.018364,[],1.0,0.0,0.0,0.0,0.0,2.078836e+07,...,159.0,0.0,0.0,227440.0,194.0,342.0,96.0,72.0,0.0,None
4,isaplanner,4.0,0.026222,[],1.0,0.0,0.0,0.0,0.0,3.497952e+07,...,327.0,0.0,0.0,421480.0,359.0,701.0,280.0,148.0,0.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,thread 'main' panicked at 'called `Result::unw...
241,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,thread 'main' panicked at 'called `Result::unw...
242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,thread 'main' panicked at 'called `Result::unw...
243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Timeout Exception


In [179]:
print(jsons['case_split_proof_split_4_split_colored_stats']['error'].value_counts())

error
Timeout Exception                                                                                                                                                                                                                                                                                                                                  63
08:34:37 [ERROR] Unrecognized token `=>` found at 3771:3773\nExpected one of "(", "=", "?", r#"#[^#]+#"# or r#"[a-zA-Z+\\-/_*&$%^@0-9]+"#\nthread 'main' panicked at 'Please implement error handling', src/thesy/semantics.rs:123:17\nnote: run with `RUST_BACKTRACE=1` environment variable to display a backtrace\n                18
thread 'main' panicked at 'called `Result::unwrap()` on an `Err` value: UnrecognizedToken { token: (1, Token(13, "=>"), 3), expected: ["\"(\"", "\"=\"", "\"?\"", "r#\"#[^#]+#\"#", "r#\"[a-zA-Z+\\\\-/_*&$%^@0-9]+\"#"] }', src/bin/expl_experiment.rs:127:34\nnote: run with `RUST_BACKTRACE=1` environment variable to display a

In [180]:
j = list(jsons.values())[0]
print(j.columns)
j.describe()

Index(['test_suite', 'Unnamed: 1', 'time', 'stop_reasons', 'success',
       'lemma_count', 'proofs_later_filtered', 'case_split_root_count',
       'case_split_had_vacuity', 'total_allocated', 'max_allocated',
       'file_name', 'relative_path', 'split_depth', 'features',
       'num_terms_added', 'total_search_time', 'total_apply_time',
       'total_rebuild_time', 'total_enodes', 'num_splits', 'should_delete',
       'deleted_colored_enodes', 'colored_unions_count', 'total_colors_size',
       'black_size', 'vacuos_colors', 'split_sizes', 'graph_memory',
       'eclasses_count', 'id_count', 'colored_equivalences_size',
       'black_colored_classes_size', 'parent_color_tracking_size', 'error'],
      dtype='object')


,Unnamed: 1,time,lemma_count,proofs_later_filtered,case_split_root_count,total_allocated,max_allocated,split_depth,num_terms_added,total_search_time,...,total_colors_size,black_size,vacuos_colors,split_sizes,graph_memory,eclasses_count,id_count,colored_equivalences_size,black_colored_classes_size,parent_color_tracking_size
count,107.000000,107.000000,107.0,107.0,107.0,1.070000e+02,1.070000e+02,107.0,107.000000,107.000000,...,107.0,107.000000,107.0,107.0,1.070000e+02,107.000000,107.000000,1.070000e+02,107.0,107.0
mean,25.691589,1.962168,0.0,0.0,0.0,1.689585e+09,7.822505e+06,4.0,56.401869,0.000275,...,0.0,382.130841,0.0,0.0,3.754808e+06,245.710280,389.355140,5.850841e+04,0.0,0.0
std,19.581186,18.698518,0.0,0.0,0.0,1.589556e+10,6.542584e+07,0.0,33.981921,0.000417,...,0.0,1227.276891,0.0,0.0,3.337768e+07,878.379576,1243.507283,5.519260e+05,0.0,0.0
min,0.000000,0.002097,0.0,0.0,0.0,9.496956e+06,4.437860e+05,4.0,2.000000,0.000041,...,0.0,41.000000,0.0,0.0,2.083200e+04,37.000000,41.000000,0.000000e+00,0.0,0.0
25%,8.500000,0.003726,0.0,0.0,0.0,1.047661e+07,4.554860e+05,4.0,20.500000,0.000105,...,0.0,91.000000,0.0,0.0,4.494200e+04,72.500000,91.000000,0.000000e+00,0.0,0.0
50%,22.000000,0.004712,0.0,0.0,0.0,1.099879e+07,4.613990e+05,4.0,52.000000,0.000175,...,0.0,130.000000,0.0,0.0,7.362800e+04,121.000000,131.000000,0.000000e+00,0.0,0.0
75%,40.500000,0.011264,0.0,0.0,0.0,1.728292e+07,5.208540e+05,4.0,87.000000,0.000284,...,0.0,163.500000,0.0,0.0,9.692400e+04,145.500000,163.500000,9.000000e+00,0.0,0.0
max,67.000000,193.437934,0.0,0.0,0.0,1.644347e+11,6.761668e+08,4.0,124.000000,0.003665,...,0.0,10770.000000,0.0,0.0,3.435838e+08,8463.000000,10929.000000,5.695054e+06,0.0,0.0


In [181]:
# Write our all new csv files as updated_stats.csv
for n, j in jsons.items():
    j.to_csv(results_root / (n[23:] + "_updated_stats.csv"))

master_df = pd.concat(jsons.values())
master_df.to_csv(results_root / "master_updated_stats.csv")